In [2]:
import socket,sys,os
from filelock import FileLock

sock=socket.socket(socket.AF_INET,socket.SOCK_STREAM)
server_address=("localhost",6744)
sock.bind(server_address)


print("TCP server is up")
seq=1
while True:
    sock.listen(1)
    connection, client_address= sock.accept()
    print("Connection with client {}".format(client_address))
    while True:
        print("Waiting for command..")
        data=connection.recv(1024)
        print(data.decode())

        if "get" in data.decode() and seq==2:
            print(data.decode())
            name=data.decode().replace("get ","")
            if name in os.listdir("."):
    
                #lock.acquire()
                file=open(name,"br")
                
                connection.sendall(file.read())
                #print(file.read().decode())
                file.close()
                print("Done sending")
                seq=3

            else: 
                connection.sendall("Error, the file does not exist".encode())
                print("The file the client asked for does not exist")

        elif "put" in data.decode() and seq==3:
            name=data.decode().replace("put ","")
            route= name
            connection.sendall("Put OK".encode())
            with open(route,"bw") as file:
                data=connection.recv(1024)
                file.write(data)
                #print(data.decode())
                file.close()
                print("Done modifying")
                seq=4
            

        elif "lock" in data.decode() and seq==1:
            name=data.decode().replace("lock ","")
            print("Legamos al lock")
            if name in os.listdir():
                route= name
                lock_route=route+".lock"
                lock=FileLock(lock_route)
                lock.acquire()
                connection.sendall("Lock OK".encode())
                seq=2
                
            else: 
                connection.sendall("Error, the file does not exist".encode())
                print("The file the client asked for does not exist")
                break
            
        
        elif "unlock" in data.decode() and seq==4:
            name=data.decode().replace("unlock ","")
            route="../server_files/" + name
            lock.release()
            print("Done Unlocking")
            seq=1
            connection.sendall("Done unlocking".encode())

        elif "bye" in data.decode():
            connection.sendall("Bye Client".encode())
            print("Bye Client")
            try:
                lock.release()
            except:
                print("Unable to unlock file")
            finally:
                
                connection.close()
                break
        
        else:
            connection.sendall("Error, introduce the commands in order: lock,get,put,unlock or exit sending 'bye'".encode())
            





TCP server is up
Connection with client ('127.0.0.1', 51162)
Waiting for command..
put f1.txt
Waiting for command..
lock f1.txt
Legamos al lock
Waiting for command..
get f1.txt
get f1.txt
Done sending
Waiting for command..
put f1.txt
Done modifying
Waiting for command..
unlock f1.txt
Done Unlocking
Waiting for command..
bye
Bye Client


KeyboardInterrupt: 